In [ ]:
import json
import redis
from flask import Flask, request
from loguru import logger

HISTORY_LENGTH = 10
DATA_KEY = "engine_temperature"


In [ ]:
@app.route('/record', methods=['POST'])
def record_engine_temperature():
    payload = request.get_json(force=True)
    logger.info(f"(*) record request --- {json.dumps(payload)} (*)")

    engine_temperature = payload.get("engine_temperature")
    logger.info(f"engine temperature to record is: {engine_temperature}")

    database = redis.Redis(host="redis", port=6379, db=0, decode_responses=True)
    database.lpush(DATA_KEY, engine_temperature)
    logger.info(f"stashed engine temperature in redis: {engine_temperature}")

    while database.llen(DATA_KEY) > HISTORY_LENGTH:
        database.rpop(DATA_KEY)
    engine_temperature_values = database.lrange(DATA_KEY, 0, -1)
    logger.info(f"engine temperature list now contains these values: {engine_temperature_values}")

    logger.info(f"record request successful")
    return {"success": True}, 200


In [ ]:
docker-compose down
docker-compose up


In [ ]:
import requests

response = requests.post("http://0.0.0.0:8000/record", json={"engine_temperature": 0.3})
print(response.json())


In [ ]:
{
  "engine_temperature": 0.3
}


In [ ]:
curl -X POST http://0.0.0.0:8000/record -H "Content-Type: application/json" -d '{"engine_temperature": 0.3}'


In [ ]:
git add .
git commit -m "Integrated Redis for engine temperature recording"
git push origin main
